<a href="https://colab.research.google.com/github/RVasconcelosMiguel/ComputerVision/blob/main/CV_Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
pip install tqdm

In [13]:
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import numpy as np
import random
from tqdm import tqdm  # Importar tqdm

import os
import pandas as pd
import json

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
from torch.autograd import Variable
from torch.optim import Adam, SGD, NAdam
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler


import torchvision
from torchvision import models, transforms
from torchvision.io import read_image

from torchsummary import summary

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, label_binarize
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from PIL import Image
import zipfile

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("The model will be running on", device, "device")

The model will be running on cpu device


In [15]:
import zipfile

#Limpa arquivos existentes do sistema antes de iniciar as operações.
!rm -f -r "images"
!rm -f "module_metadada.json"

#Definição de caminhos
download_path = "/content/InfraredSolarModules/"
download_path_file = "/content/InfraredSolarModules/2020-02-14_InfraredSolarModules.zip"
zip_path = "/content/2020-02-14_InfraredSolarModules.zip"
extract_path = "/content"

# Clonar o repositorio e fazer unzip
!git clone https://github.com/RaptorMaps/InfraredSolarModules.git $download_path
!mv $download_path_file $extract_path
!unzip -uq $zip_path -d $extract_path

# Imagens
image_folder_path = "/content/InfraredSolarModules/images"
!mv $image_folder_path $extract_path

# module_metadata.json
metadata_file_path = "/content/InfraredSolarModules/module_metadata.json"
!mv $metadata_file_path $extract_path

#Remoção de arquivos desnecessários
!rm -f $zip_path
!rm -f -r $download_path
!rm -r "__MACOSX"

Cloning into '/content/InfraredSolarModules'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 24 (delta 10), reused 12 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (24/24), 5.49 MiB | 15.60 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [16]:
classes_a = ("anomaly", "no-anomaly")
classes_b=("cell","cell-multi","cracking","hotspot","hot-spot-multi","shadowing","diode","diode-multi","vegetation","soiling","offlinemodule")
classes_c=("cell","cell-multi","cracking","hotspot","hot-spot-multi","shadowing","diode","diode-multi","vegetation","soiling","offlinemodule","no-anomaly")


metadata_path = '/content/module_metadata.json'
with open(metadata_path, 'r') as f:
    metadata = json.load(f)

metadata_df_a = pd.DataFrame.from_dict(metadata, orient='index')
metadata_df_b = pd.DataFrame.from_dict(metadata, orient='index')
metadata_df_c = pd.DataFrame.from_dict(metadata, orient='index')

metadata_df_a['anomaly_class'] = metadata_df_a['anomaly_class'].apply(lambda x: 'Anomaly' if x != 'No-Anomaly' else 'No-Anomaly')
metadata_df_b = metadata_df_b[metadata_df_b["anomaly_class"] != "No-Anomaly"]

print("---------a) dataframe------------")
print("\n Data \n",metadata_df_a.tail(10))
print("\n Data size :",metadata_df_a.shape[0])
print("\n")

print("---------b) dataframe------------")
print("\n Data \n",metadata_df_b.tail(10))
print("\n Data size :",metadata_df_b.shape[0])
print("\n")

print("---------c) dataframe------------")
print("\n Data \n",metadata_df_c.tail(10))
print("\n Data size :",metadata_df_c.shape[0])
print("\n")

---------a) dataframe------------

 Data 
          image_filepath anomaly_class
8483    images/8483.jpg       Anomaly
8484    images/8484.jpg       Anomaly
8485    images/8485.jpg       Anomaly
8486    images/8486.jpg       Anomaly
8487    images/8487.jpg       Anomaly
8488    images/8488.jpg       Anomaly
8489    images/8489.jpg       Anomaly
7464    images/7464.jpg       Anomaly
18065  images/18065.jpg    No-Anomaly
13354  images/13354.jpg    No-Anomaly

 Data size : 20000


---------b) dataframe------------

 Data 
        image_filepath anomaly_class
8481  images/8481.jpg    Vegetation
8482  images/8482.jpg    Vegetation
8483  images/8483.jpg    Vegetation
8484  images/8484.jpg    Vegetation
8485  images/8485.jpg    Vegetation
8486  images/8486.jpg    Vegetation
8487  images/8487.jpg    Vegetation
8488  images/8488.jpg    Vegetation
8489  images/8489.jpg    Vegetation
7464  images/7464.jpg      Cracking

 Data size : 10000


---------c) dataframe------------

 Data 
          imag